In [1]:
!pip install jieba

In [3]:
import json
import os
import sys
import urllib.request
import copy
import csv
#from google.colab import drive
import jieba as j
import pandas as pd

#번역함수 -> (번역할 문장, 번역할 언어종류)
def translate(sentence, target_lang):
    client_id = "deribthgxo"
    client_secret = "8NOoY9KhtwKHKZwpOQYr5bovSKA6DSctcC9eClf8"
    encText = urllib.parse.quote(sentence)
    data = f"source=ko&target={target_lang}&text=" + encText

    url = "https://naveropenapi.apigw.ntruss.com/nmt/v1/translation"
    request = urllib.request.Request(url)
    request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
    request.add_header("X-NCP-APIGW-API-KEY", client_secret)
    response = urllib.request.urlopen(request, data=data.encode("utf-8"))
    rescode = response.getcode()

    if (rescode == 200):
        response_body = response.read()
        result = response_body.decode('utf-8')
        json_sentence = json.loads(result)
        return json_sentence["message"]["result"]["translatedText"]

    else:
        return "Error Code:" + rescode


#Data 로딩 함수 -> (csv파일 경로, 몇번째 row까지 사용할 것인지 ...)
def data_reload(csv_path, start_index, end_index):

    data = []
    with open(csv_path, 'r', encoding='utf-8') as csvfile:
        data_reader = csv.reader(csvfile)
        for row in data_reader:
            data.append(row)

    del data[0] #컬럼 이름이 들어간 row는 del메소드로 삭제해준다.

    data = data[start_index : end_index]

    return data

#불용어 추출 함수 -> (받아온 데이터, 불용어 추출 리스트)
def StopWordsFiltering(data, stopwords):
    for i, row in enumerate(data):
        elem1 = ''.join(row[0])
        elem2 = ''.join(row[1])
        for s in stopwords:
            row1 = elem1.replace(s, "")
            row2 = elem2.replace(s, "")
        # row1 = elem1.lstrip().rstrip().replace("ㅎ", "").replace("~", "").replace("ㅠ", "").replace("휴", "").replace("우", "").replace("ㅜ", "")
        # row2 = elem2.lstrip().rstrip().replace("ㅎ", "").replace("~", "").replace("ㅠ", "").replace("휴", "").replace("우", "").replace("ㅜ", "")
        data[i][0] = row1
        data[i][1] = row2 
    print(data[:5])
    
    return data

#앞에서 받아온 데이터를 forloop로 일일히 번역해주는 함수! -> (번역할 데이터)
def Translate_UsingPAPAGO(trans_data, default_lang): 
    Q_list = []
    A_list = []
    label = []

    for i, row in enumerate(trans_data):
        Q_list.append(translate(row[0], default_lang))
        A_list.append(translate(row[1], default_lang))
        label.append(row[2])
        print(f"complete-row{i}!")

    for i, j, k in zip(Q_list[:5], A_list[:5], label[:5]):
        print(i, j, k)

    return Q_list, A_list, label

#jieba모듈을 사용해서 문장을 토크나이징 해주는 함수 -> (Q, A, label컬럼에 해당하는 데이터)
def jieba_tokenizing(Q_list, A_list, label):
    j.enable_paddle()

    input_data = {
        "Q" : [" ".join(j.cut(q, cut_all=False, use_paddle=True)) for q in Q_list],
        "A" : [" ".join(j.cut(a, cut_all=False, use_paddle=True)) for a in A_list],
        "Label" : [" ".join(j.cut(l, cut_all=False, use_paddle=True)) for l in label]
    }

    new_df = pd.DataFrame(input_data)

    print(new_df.head(3))

    return new_df

#Main함수
if __name__ == "__main__":
    csv_path = r"C:\Users\WIN10\Desktop\UCCProject\ChineseChatBotProject\chaotbotproject\chatbot\model\chinense_sample_data.xlsx"
    
    stopwords = ["ㅎ", "~", "'", "*", "^", "휴", "우", "후", "ㅠ", "ㅜ", "."]
    #데이터 임포트
    data = data_reload(csv_path, 0, 10) #총 row개수 -> 11824개 (준기[0:3943] 영석[3943:7883] 상민[7883:-1])
    #데이터 스탑워드 필터링 !
    filtering_data = StopWordsFiltering(data, stopwords)
    print(filtering_data)
    #번역함수 돌리기!
    Q_list, A_list, label = Translate_UsingPAPAGO(filtering_data,  "zh-CN")
    #jieba 토크나이징 진행하기 !
    new_df = jieba_tokenizing(Q_list, A_list, label)
    new_df.to_excel("chinense_chatbot_data.xlsx")

UnicodeDecodeError: 'cp949' codec can't decode byte 0xf2 in position 10: illegal multibyte sequence